In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from zipfile import ZipFile
file_names = ['/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip','/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip']
for i in file_names:
    with ZipFile(i, 'r') as zip:
        zip.printdir()
        zip.extractall()

In [ ]:
train = pd.read_csv('train.csv') 
y_train = train['type']
x_train = train
y_train = pd.DataFrame(y_train)
y_train = pd.get_dummies(y_train)

In [ ]:
train['type'].value_counts()

In [ ]:
test = pd.read_csv('test.csv') 
id_test = test['id']
test = test.drop(columns=['id'])

In [ ]:
colors = x_train['color'].unique()
replace_colors = np.array(range(colors.shape[0]))/(colors.shape[0]-1)

for i in range(x_train['color'].shape[0]):
    for j in range(colors.shape[0]):
        if x_train['color'][i] == colors[j]:
            x_train['color'][i] = replace_colors[j]
x_train["color"] = x_train["color"].astype(float)

In [ ]:
for i in range(test['color'].shape[0]):
    for j in range(colors.shape[0]):
        if test['color'][i] == colors[j]:
            test['color'][i] = replace_colors[j]
test["color"] = test["color"].astype(float)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train)
X_train,y_train = x_train, y_train
X_train = X_train.drop(columns=['type','id'])

In [ ]:
#test_id = X_test['id']
#X_train, X_test = X_train.drop(columns=['type','id']), X_test.drop(columns=['type','id'])

In [ ]:
print(X_train.shape)
print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25, batch_size=32)

In [ ]:
pred = model.predict(test)

In [ ]:
#y_test = np.array(y_test)

In [ ]:
list_pred = []
for i in range(pred.shape[0]):
    if max(pred[i]) == pred[i][0]:
        list_pred.append('type_Ghost')
    elif max(pred[i]) == pred[i][1]:
        list_pred.append('type_Ghoul')
    else:
        list_pred.append('type_Goblin')

In [ ]:
submission = pd.DataFrame({'id':id_test, 'type':list_pred})
submission.head()

In [ ]:
submission.to_csv('../working/submission.csv', index=False)